In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

1. ¿Qué tipos de prestación médica son más frecuentes entre diferentes tramos de edad y géneros?

2. ¿Cómo varía la cantidad y tipo de prestaciones emitidas según el mes del año? ¿Existen patrones estacionales?

3. ¿Qué tipos de prestación médica son más frecuentes en cada región?

4. ¿Cómo se relacionan los distintos tramos de Fonasa con la frecuencia y el tipo de prestaciones recibidas?

5. ¿Cómo cambia el gasto promedio en copagos a medida que los beneficiarios envejecen?